# Basic statutory expert system 3 — case law

* Introduce the ```Case```, ```Journal```, and ```Offence``` classes and sort out how they relate to each other
    * Inter-object dependencies
    * Hierarchical relationships
* Move the high treason functions into an ```Offence``` instance
* Create a ```Journal``` instance
* Create a ```Case``` instance
* 

## Models

The first step in creating an ```Offence``` class is to determine what's needed to accommodate the offence framework currently contained in the ```main.py``` functions. These include:

* ```HIGH_TREASON_RULES```
* ```high_treason_facts()```
* ```high_treason_rules()``` 
* ```verify_high_treason()```

### ```HIGH_TREASON_RULES```

This constant contains both the questions needed to create the factual matrix required and indexable values associated with the answers to those questions. ```high_treason_rules()``` reads those values. More basically, the factual matrix can and should be thought of as the offence elements that, when present in the correct pattern, make out the offence. 

### ```high_treason_facts()```

Currently a function called by the ```create_facts()``` method, ```high_treason_facts()``` checks to see whether the complainant is either Canada or the sovereign, as high treason can only be committed against one or the other. 

#### ```get_sovereign_actions()```

If the complainant is the sovereign, this function retrieves the actions associated with high treason against the sovereign.

#### ```get_canada_actions()```

If the complainant is Canada, this function retrieves the actions associated with high treason against Canada.

### ```high_treason_rules()```


### New design

* Combine ```high_treason_facts()``` & ```get_sovereign_actions()``` & ```get_canada_actions()``` into a single UI process
* Design a dictionary for offence elements that can be adapted to other criminal offences
* 

In [3]:
# Unified treason_offence_elements() function

def treason_offence_elements(victim_category) -> list:
    """
    Unified UI
    """
    
    actions = []
    
    if victim_category.lower() == 'sovereign':
        for question in HIGH_TREASON_RULES["sovereign"]:
            response = input(question[0] + " (yes/no): ")
            if response.lower() == 'yes':
                actions.append(("sovereign", question[1]))
        return actions
        
    elif victim_category.lower() == 'canada':
        for question in HIGH_TREASON_RULES["canada"]:
            response = input(question[0] + " (yes/no): ")
            if response.lower() == 'yes':
                actions.append(("canada", question[1]))
        return actions

    else:
        return None


### Offence elements

The offence dictionary's primary purpose is to store an offence's elements. 

Some offence elements will be present in any Canadian criminal offence, such that a criminal offence occurs only iff they are present. These include:

* Date/time
* Jurisdiction
* Identity
* Liability type
* Statutory authority

Other offence elements will only be present in certain offences, and may not all be required to be present to make out the offence. Where that is the case, it is often also the case that more than one of these offence elements will mean that more than one instance of the offence exists. High treason, for example, includes several elements that aren't present in other Canadian criminal offence, including:

* Doing bodily harm that tends to cause death or destruction
* Restraining the sovereign
* Imprisoning the sovereign
* Assisting an enemy at war with Canada

Any one of these elements is enough to make out high treason. If more than one is present, it may be the case that more than one offence has been committed, provided there are separate transactions. I can start implementing code to identify, distinguish, and handle these separate transactions once the expert system is more completely laid out. Until then, it will suffice to say that each ```Offence``` instance represents a separate transaction. Where transactions contain multiple offence possibilities, they will be inclusively contained and subject to *Kienapple* if more than one is made out.

In order to be usefully implemented into a chat model like GPT, these offence elements will need to be broken down into the smallest possible atoms. For example, the offence element "Did the defendant do bodily harm to the sovereign tending to cause death or destruction?" can and should be broken down further into the following atomic elements:

* Death
    * The defendant caused bodily harm
    * The sovereign suffered the bodily harm
    * The bodily harm the sovereign suffered tends to cause death
* Destruction
    * The defendant caused bodily harm
    * The sovereign suffered the bodily harm
    * The bodily harm the sovereign suffered tends to cause destruction
    
Elements can be necessary, sufficient, or both.

All offence elements can be expressed as tuples. The first index contains the factual matrix while the second describes the element's relationship to the offence generally. 

Universal elements should be evaluated separately from existential elements. Because universal elements are always required, they can be evaluated first to see if the facts support the charge past these basics. If so, the existential elements can be considered next.

The necessity or sufficiency of the existential elements assumes that the requisite universal elements are made out. That is, an existential element may be necessary and sufficient, but only if all the universal elements are met. If they are not, the existential element will not be sufficient.

Some elements are compounds. 

Designing the offence dictionary as a dictionary allows me to export and store it as a JSON file.

In [9]:
# Offence template

offence_dictionary = {
    "universal elements": {
        "datetime": (None, ["necessary"]),
        "jurisdiction": (None, ["necessary"]),
        "identity": (None, ["necessary"]),
        "liability": (None, ["necessary"]),
        "statute": None
    },
    "existential elements": {
    }
}


# Copy "universal elements" from the "offence_dictionary" to create a specific instance

high_treason_dictionary = {
    "universal elements": offence_dictionary["universal elements"].copy(),
    "existential elements": {
        
    }
}


In [8]:
class Offence:
    
    def __init__(self,
                 offence_name = None,
                 offence_section = None,
                 punishment = None,
                 punishment_section = None,
                 offence_text = None,
                 offence_elements = None,
                 available_defences = None,
                 ):
        
        self.offence_name = offence_name
        self.offence_section = offence_section
        self.punishment = punishment
        self.punishment_section = punishment_section
        self.offence_text = offence_text
        self.offence_elements = offence_elements
        
    # Offence elements
        
        
        
        

In [37]:
cc_s_2_definitions = {
    "act": 
    (("Act includes\n\t(a) an Act of Parliament,"
      "\n\t(b) an Act of the legislature of the former Province of Canada,"
      "\n\t(c) an Act of the legislature of a province, and"
      "\n\t(d) an Act or ordinance of the legislature of a province, territory"
      " or place in force at the time that province, territory or place became"
      "a province of Canada."
     ), 
     ("S’entend notamment :\n\ta) d’une loi fédérale;"
     
     )\n\tb) d’une loi de la législature d l’ancienne province du Canada;\n\tc) d’une loi provinciale;\n\td) d’une loi ou ordonnance de la législature d’une province, d’un territoire ou d’un endroit, en vigueur au moment où cette province, ce territoire ou cet endroit est devenu une province du Canada.", "loi"),
    "appearance notice": 
    ("appearance notice means a notice in Form 9 issued \
    by a peace officer.", "Citation selon la formule 9, délivrée par un agent \
    de la paix.", "citation à comparaître"),
    "associated personnel": 
    ("associated personnel means persons who are\n\t\
    (a) assigned by a government or an intergovernmental organization with the\
     agreement of the competent organ of the United Nations,\n\t(b) engaged by\
     the Secretary-General of the United Nations, by a specialized agency of \
     the United Nations or by the International Atomic Energy Agency, or\n\t\
    (c) deployed by a humanitarian non-governmental organization or agency \
    under an agreement with the Secretary-General of the United Nations, by a \
    specialized agency of the United Nations or by the International Atomic \
    Energy Agency,\nto carry out activities in support of the fulfilment of \
    the mandate of a United Nations operation", "Les personnes ci-après qui \
    exercent des activités dans le cadre d’une opération des Nations Unies :\
    \n\ta) les personnes affectées par un gouvernement ou une organisation \
    intergouvernementale avec l’accord de l’organe compétent de l’Organisation\
     des Nations Unies;\n\tb) les personnes engagées par le Secrétaire général\
     de l’Organisation des Nations Unies, par une institution spécialisée de \
    cette organisation ou par l’Agence internationale de l’énergie atomique; \
    \n\tc) les personnes affectées par une organisation non gouvernementale \
    humanitaire en vertu d’un accord avec le Secrétaire général de \
    l’Organisation des Nations Unies, par une institution spécialisée de cette\
     organisation ou par l’Agence internationale de l’énergie atomique.", 
    "personnel associé"),
    "attorney general": 
    ("(a) with respect to proceedings to which this Act applies, means the \
    Attorney General or Solicitor General of the province in which those \
    proceedings are taken and includes his or her lawful deputy or, if those \
    proceedings are referred to in subsection 2.3(1), the Attorney General of \
    Canada or the Attorney General or Solicitor General of the province in \
    which those proceedings are taken and includes the lawful deputy of any of\
     them,\n\t(b) means the Attorney General of Canada and includes his or her\
     lawful deputy with respect to\n\t\t(i) Yukon, the Northwest Territories \
    and Nunavut, or\n\t\t(ii) proceedings commenced at the instance of the \
    Government of Canada and conducted by or on behalf of that Government in \
    respect of an offence under any Act of Parliament — other than this Act or\
     the Canada Elections Act — or any regulation made under such an Act, and\
    \n\t(c) means the Director of Public Prosecutions appointed under \
    subsection 3(1) of the Director of Public Prosecutions Act with respect to\
     proceedings in relation to an offence under the Canada Elections Act;",),
    
    
}